In [1]:
!pip install tensorflow==2.12 keras==2.12 larq

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.3 MB/s eta 0:00:00
   ━━━

In [1]:
import tensorflow as tf
import larq as lq
import numpy as np
import os
import shutil

# Loading and preprocessing MNIST data
(_, _), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
test_images = test_images.reshape(-1, 784).astype("float32") / 127.5 - 1
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

11490434/11490434 [==============================] - 1s 0us/step


In [2]:
# Loading the trained model
model = tf.keras.models.load_model("model_trained/binarized_model")

# Compiling the model before evaluation
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [3]:
# Function to binarize weights
def get_binarized_weights(layer):
    weights = layer.get_weights()[0]
    quantizer = layer.kernel_quantizer
    return quantizer(weights).numpy()

In [4]:
# Printing binarized weights before row inversion
print("\nBinarized Weights Before Row Inversion:")
for layer in model.layers:
    if isinstance(layer, lq.layers.QuantDense):
        print(f"\nLayer: {layer.name}")
        print(get_binarized_weights(layer)[:10, :10])


Binarized Weights Before Row Inversion:

Layer: quant_dense
[[-1.  1.  1.  1.  1.  1. -1. -1. -1.  1.]
 [ 1. -1. -1. -1.  1.  1.  1.  1. -1. -1.]
 [-1. -1.  1. -1. -1. -1.  1.  1.  1. -1.]
 [ 1. -1. -1. -1. -1. -1.  1.  1.  1. -1.]
 [ 1. -1.  1.  1.  1.  1. -1. -1. -1.  1.]
 [-1.  1.  1. -1. -1.  1.  1. -1. -1.  1.]
 [ 1. -1. -1. -1. -1. -1.  1.  1. -1.  1.]
 [ 1.  1.  1. -1.  1.  1.  1. -1. -1.  1.]
 [-1.  1. -1.  1. -1.  1.  1.  1.  1. -1.]
 [ 1. -1. -1.  1.  1.  1. -1. -1. -1.  1.]]

Layer: quant_dense_1
[[ 1. -1. -1.  1.  1. -1.  1.  1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1.  1. -1.  1.  1.  1. -1. -1.]
 [ 1.  1.  1.  1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1.  1. -1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1.  1. -1. -1. -1.  1.  1. -1.]
 [-1.  1. -1.  1. -1. -1. -1.  1. -1.  1.]
 [-1.  1. -1. -1. -1. -1.  1. -1. -1. -1.]
 [-1. -1.  1.  1. -1.  1.  1.  1.  1. -1.]
 [ 1. -1.  1.  1. -1.  1. -1. -1.  1. -1.]]

Layer: quant_dense_2
[[ 1. -1.  1. -1. -1.  1.  1. -1.

In [5]:
# Row inversion function
def apply_row_inversion(model):
    for layer in model.layers:
        if isinstance(layer, lq.layers.QuantDense):
            weights = layer.get_weights()
            if weights:
                W = weights[0]
                rows = W.shape[0]
                row_puf = np.random.randint(0, 2, size=rows)
                W_new = np.copy(W)
                for r in range(rows):
                    if row_puf[r] == 1:
                        W_new[r, :] *= -1
                layer.set_weights([W_new])
    print("\nRow Inversion Applied.")

In [6]:
# Applying row inversion
apply_row_inversion(model)


Row Inversion Applied.


In [7]:
# Printing binarized weights after row inversion
print("\nBinarized Weights After Row Inversion:")
for layer in model.layers:
    if isinstance(layer, lq.layers.QuantDense):
        print(f"\nLayer: {layer.name}")
        print(get_binarized_weights(layer)[:10, :10])


Binarized Weights After Row Inversion:

Layer: quant_dense
[[ 1. -1. -1. -1. -1. -1.  1.  1.  1. -1.]
 [ 1. -1. -1. -1.  1.  1.  1.  1. -1. -1.]
 [-1. -1.  1. -1. -1. -1.  1.  1.  1. -1.]
 [-1.  1.  1.  1.  1.  1. -1. -1. -1.  1.]
 [-1.  1. -1. -1. -1. -1.  1.  1.  1. -1.]
 [ 1. -1. -1.  1.  1. -1. -1.  1.  1. -1.]
 [-1.  1.  1.  1.  1.  1. -1. -1.  1. -1.]
 [-1. -1. -1.  1. -1. -1. -1.  1.  1. -1.]
 [-1.  1. -1.  1. -1.  1.  1.  1.  1. -1.]
 [-1.  1.  1. -1. -1. -1.  1.  1.  1. -1.]]

Layer: quant_dense_1
[[ 1. -1. -1.  1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1. -1. -1.]
 [-1. -1. -1. -1.  1. -1. -1. -1.  1.  1.]
 [-1. -1. -1. -1.  1.  1.  1.  1. -1. -1.]
 [-1. -1. -1. -1.  1. -1.  1.  1. -1. -1.]
 [-1. -1. -1. -1.  1.  1.  1. -1. -1.  1.]
 [ 1. -1.  1. -1.  1.  1.  1. -1.  1. -1.]
 [-1.  1. -1. -1. -1. -1.  1. -1. -1. -1.]
 [ 1.  1. -1. -1.  1. -1. -1. -1. -1.  1.]
 [ 1. -1.  1.  1. -1.  1. -1. -1.  1. -1.]]

Layer: quant_dense_2
[[-1.  1. -1.  1.  1. -1. -1.  1. 

In [8]:
# Evaluating the inverted model
print("\nModel Accuracy After Row Inversion:")
inv_loss, inv_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy After Row Inversion: {inv_acc * 100:.2f}%")


Model Accuracy After Row Inversion:
313/313 [==============================] - 2s 6ms/step - loss: 2.4620 - accuracy: 0.1004
Test Accuracy After Row Inversion: 10.04%


In [9]:
# Saving the inverted model
os.makedirs("model_row_inverted", exist_ok=True)
model.save("model_row_inverted/binarized_model_row_inverted", include_optimizer=False)

# Zip the saved model directory
shutil.make_archive("model_row_inverted", 'zip', 'model_row_inverted')

'/content/model_row_inverted.zip'